# Was wordt lijst

Hoofdvraag: welke landelijke typologie (SBB-code, VvN of rVvN) wordt onze standaard die we als input voor Veg2Hab gebruiken en waarom?
- rVvN is het nieuwst, en beschrijft als het goed is de meeste vegetatietypen. gecheckt: als iets een SBB of VvN code heeft, heeft het ook altijd een rVvN code. Andersom niet.
- De opdrachtgever gaf als commentaar terug dat zij vinden dat VvN de standaard moet zijn.

#### Openstaande vragen voor Johannes / begeleidingscommissie:
- Waarom bevat kolom `rVvN` waardes 'Niet'? Moeten we die op NA zetten?
- Waarom staan sommige waardes in kolom `VvN` tussen haakjes? 
- Waarom wil opdrachtgever dat VvN de standaard input voor Veg2Hab is, en niet het nieuwere rVvN?
- Vanaf rij 989 bevat VvN geen waardes meer. rVvN wel, en sommige ook een SBB-code. Waarom geen VvN?
- Wat is de 17e kolom?

#### Beantwoorde vragen:

**Vraag**:

**Antwoord:**

## Data inladen

In [3]:
import pandas as pd

In [36]:
df = pd.read_excel('../data/5. Was-wordt-lijst-vegetatietypen-en-habitattypen-09-02-2021.xlsx')

/home/jakko/git/veg2hab/.venv/lib/python3.8/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


In [38]:
df.columns

Index(['rVvN', 'Wetenschappelijke Naam rVvN', 'Nederlandse Naam rVvN',
       'Veldgids RG', 'VvN', 'SBB-code', 'SBB Wet. Naam',
       'Opmerking vertaling',
       'Habitattypen (blauw: geldige aanvulling; rood: voorstel; zie Toelichting)',
       'discussiepunten Wwlijst', 'actie nodig', 'principe', 'impact',
       'discussie/huiswerk', 'wie', 'uitkomst', 'openstaand 27-11-2018',
       'Unnamed: 17'],
      dtype='object')

In [60]:
ww = df[['rVvN','VvN','SBB-code', 'Habitattypen (blauw: geldige aanvulling; rood: voorstel; zie Toelichting)']].rename(columns={'Habitattypen (blauw: geldige aanvulling; rood: voorstel; zie Toelichting)':'Habtype'})

## Volledigheid typologie

rVvN lijkt de meest specifieke vegetatietypologie. ALles met een waarde voor SBB of VvN heeft altijd (1 uitzondering, zie beneden) ook een rVvN code, andersom niet. 

Velden met een SBB code hebben altijd ook een habitattype ingevuld, velden met VvN of rVvN niet.

In [61]:
ww.isna().sum()

rVvN          5
VvN         257
SBB-code    352
Habtype     211
dtype: int64

In [62]:
# Rijen met een waarde voor rVvN hebben NIET altijd een waarde voor VvN of SBB
ww[~ww['rVvN'].isna()].isna().sum()

rVvN          0
VvN         252
SBB-code    348
Habtype     207
dtype: int64

In [63]:
# Rijen met een waarde voor VvN hebben WEL altijd een waarde voor rVvN, maar niet voor SBB
ww[~ww['VvN'].isna()].isna().sum()

rVvN          0
VvN           0
SBB-code    224
Habtype     183
dtype: int64

In [64]:
# Rijen met een waarde voor SBB-code hebben WEL (bijna) altijd een waarde voor rVvN, maar niet voor VvN
ww[~ww['SBB-code'].isna()].isna().sum()

rVvN          1
VvN         129
SBB-code      0
Habtype       0
dtype: int64

In [48]:
ww[~ww['SBB-code'].isna() & ww['rVvN'].isna()]

,rVvN,VvN,SBB-code
1216,NaN,NaN,41A3d


In [26]:
ww['rVvN'].value_counts().value_counts()

count
1     900
2      88
3      31
5       5
4       5
19      1
6       1
Name: count, dtype: int64

In [27]:
ww['VvN'].value_counts().value_counts()

count
1    709
2     80
3     24
4      5
5      4
6      1
Name: count, dtype: int64

In [28]:
ww['SBB-code'].value_counts().value_counts()

count
1    707
3     25
2     24
4     13
5      2
Name: count, dtype: int64

In [51]:
ww[ww.duplicated(keep=False)]

,rVvN,VvN,SBB-code
416,r14Bb02,14Bb02,14D-d
417,r14Bb02,14Bb02,14D-d
1240,NaN,NaN,NaN
1241,NaN,NaN,NaN
1242,NaN,NaN,NaN
1243,NaN,NaN,NaN


## Vinden we alle waarden voor rVvN, VvN en SBB terug in de definitietabel?

Alle vegetatietypen die niet automatisch tot H0000 leiden in de was wordt, zouden we willen terugvinden in de definietabel habitattypen.

Dit is ABSOLUUT niet het geval. Voor VvN vind ik (zonder opschoningsstappen) slechts 9 van de honderden waarden terug. SBB iets meer, enkele tientallen.

In [54]:
dt = pd.read_excel('../data/definitietabel habitattypen (versie 24 maart 2009)_0.xls', sheet_name=1)

In [78]:
dt.head()

,Code vegetatietype,wetenschappelijke naam vegetatietype,Nederlandse naam vegetatietype,Code habitat (sub)type,naam habitat(sub)type,Goed / Matig,beperkende criteria,alleen in mozaïek
0,2Aa1,Ruppietum maritimae,Associatie van Snavelruppia,H1130,Estuaria,G,mits in de FGR Getijdengebied en mits gelegen ...,NaN
1,2Aa1,Ruppietum maritimae,Associatie van Snavelruppia,H1140_A,Slik- en zandplaten (getijdengebied),G,mits in de slikkige en fijnzandige delen van F...,NaN
2,2Aa1,Ruppietum maritimae,Associatie van Snavelruppia,H1160,Grote baaien,G,mits in de FGR Getijdengebied en mits in een i...,NaN
3,2Aa1,Ruppietum maritimae,Associatie van Snavelruppia,H1330_A,Schorren en zilte graslanden (buitendijks),G,NaN,alleen in mozaïek met zelfstandige vegetaties ...
4,2Aa2,Ruppietum cirrhosae,Associatie van Spiraalruppia,H1130,Estuaria,G,mits in de FGR Getijdengebied en mits gelegen ...,NaN


In [ ]:
# Ik voeg SBB- toe aan de SBB-code om de waardes te kunnen vergelijken met die uit de definitietabel

def add_SBB_prefix(x):
    if type(x) == str:
        x = 'SBB-' + x
    return x        

ww['SBB-code'] = ww['SBB-code'].apply(lambda x: add_SBB_prefix(x))

In [111]:
len(set(ww[ww['Habtype']!='H0000']['SBB-code']))

555

In [112]:
len(set(ww[ww['Habtype']!='H0000']['SBB-code']).difference(set(dt['Code vegetatietype'])))

490

In [104]:
sbb_niet_herkend = set(ww[ww['Habtype']!='H0000']['SBB-code']).difference(set(dt['Code vegetatietype']))

In [108]:
ww[ww['SBB-code'].isin(list(sbb_niet_herkend))]

,rVvN,VvN,SBB-code,Habtype
33,r46Ab03a,,SBB-43C1d,H9160A mits + H9160B mits
35,r38Aa01,(14RG10),SBB-37B1,H2160 mits
36,r37Ab01,(20RG01),SBB-20-c,H2310m + H4030m
37,r38Ab01b,(37Ac1),SBB-37-a,H2160 mits
40,r45Aa03,(42Aa01),SBB-42A1,H2180A mits + H9190 mits
...,...,...,...,...
1224,Niet overgenomen in Revisie,NaN,SBB-42-e,"toedelen tot revisie-type, anders H0000"
1226,r46Ab03b,NaN,SBB-43C1l,H9120 mits + H9160A mits + H9160B mits
1227,r46Ab03b,NaN,SBB-43C1k,H9120 mits + H9160A mits + H9160B mits
1235,r46RG11,NaN,SBB-43C1j,H9160A mits + H9160B mits
